In [1]:
%load_ext autoreload
%autoreload 2

import logging
from dotenv import load_dotenv

logging.basicConfig(level=logging.INFO)
load_dotenv()

True

In [2]:
from projects.pltv.session import get_session
from projects.pltv.feature_store import get_training_dataset

session = get_session()
sp_df = get_training_dataset(session)

INFO:snowflake.snowpark:AST state has not been set explicitly. Defaulting to ast_enabled = True.
INFO:src.connection:Creating session from private key
INFO:snowflake.connector.connection:Snowflake Connector for Python Version: 3.18.0, Python Version: 3.10.18, Platform: macOS-26.1-arm64-arm-64bit
INFO:snowflake.connector.connection:Connecting to GLOBAL Snowflake domain
INFO:snowflake.snowpark.session:Snowpark Session information: 
"version" : 1.42.0,
"python.version" : 3.10.18,
"python.connector.version" : 3.18.0,
"python.connector.session.id" : 23301353463478,
"os.name" : Darwin

INFO:root:Using database ML_LAYER
INFO:root:Creating schema PLTV
INFO:root:Using schema PLTV
INFO:src.connection:Reinstating connections from active session
INFO:snowflake.ml.feature_store.feature_store:Successfully connected to feature store: ML_LAYER.PLTV_TRAINING_FS.
INFO:src.services.feature_store_service:Spine PLTV_TRAINING SPINE created
/Users/donatodiferdinando/Documents/GitHub/ml-snowflake-template/.ve

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df = sp_df.to_pandas()

DAY_HORIZONS: list[int] = [
    30, 60, 90, 180, 365, 730
]

for day_horizon in DAY_HORIZONS:
    df[f'AVG_NET_BILLINGS_{day_horizon}_DAYS'] = df[f'NET_BILLINGS_{day_horizon}_DAYS'] / df['GROSS_ADDS']
    df.drop(columns=[f'NET_BILLINGS_{day_horizon}_DAYS'], inplace=True)
print(len(df))
df.head()



4639


,DATE_MONTH,BRAND,SKU_TYPE,CHANNEL,TRAFFIC_SOURCE,CAMPAIGN,PLAN__OFFER_TYPE,PLAN__PROMO_PRICE,PLAN__PROMO_DAYS,PLAN__RECURRING_PRICE,PLAN__RECURRING_DAYS,GROSS_ADDS,CROSS_SELLS,AVG_DAYS_TO_CROSS_SELL,AVG_CROSS_SELL_PRICE,PROMO_ACTIVATION_RATE,FIRST_REBILL_RATE,AVG_NET_BILLINGS_30_DAYS,AVG_NET_BILLINGS_60_DAYS,AVG_NET_BILLINGS_90_DAYS,AVG_NET_BILLINGS_180_DAYS,AVG_NET_BILLINGS_365_DAYS,AVG_NET_BILLINGS_730_DAYS
0,2021-11-01,TruthFinder,Background,Seo,540,unknown,standard,0.0,0,28.049999,30,1093,450,56.119999,8.088600,NaN,0.477466,39.303494,40.219526,48.822496,65.691631,86.625743,106.416035
1,2023-09-01,TruthFinder,Background,Organic,0,unknown,standard,0.0,0,28.049999,30,12711,4071,45.466961,6.495230,NaN,0.515849,40.853911,41.896522,50.897171,68.114060,89.236370,108.570293
2,2021-04-01,TruthFinder,Background,Affiliate,15,unknown,standard,0.0,0,28.049999,30,5123,921,42.243214,6.319805,NaN,0.554323,41.399336,42.049806,51.372249,68.581172,88.544377,104.138469
3,2021-01-01,TruthFinder,Background,DisplayPartner,938,unknown,standard,0.0,0,27.780001,30,701,184,73.750000,6.734022,NaN,0.586441,41.555635,41.865493,52.658574,73.577490,97.664965,115.845734
4,2023-07-01,InstantCheckmate,Background,Affiliate,648,unknown,standard,0.0,0,27.080000,30,7990,988,16.060728,4.807642,NaN,0.553683,39.733781,40.149718,49.937152,69.257814,94.823381,118.593148


In [7]:
session.write_pandas(
    df,
    'PLTV_JOINED_DATA',
    database='ML_LAYER',
    schema='PLTV',
    chunk_size=100000,
    compression='gzip',
    auto_create_table=True
)


In [ ]:
TARGET_COLS = 'AVG_NET_BILLINGS_730_DAYS'

cat_cols: list[str] = [
    'BRAND',
    'SKU_TYPE',
    'CHANNEL',
    'TRAFFIC_SOURCE',
    'CAMPAIGN',
    'PLAN__OFFER_TYPE'
]
num_cols: list[str] = [
    # 'PLAN__PROMO_PRICE',
    # 'PLAN__PROMO_DAYS',
    'PLAN__RECURRING_PRICE',
    'PLAN__RECURRING_DAYS',
    'AVG_DAYS_TO_CROSS_SELL',
    'AVG_CROSS_SELL_PRICE',
    # 'PROMO_ACTIVATION_RATE',
    'FIRST_REBILL_RATE',
]

# df[
#     cat_cols + num_cols
# ]

In [ ]:
from sklearn.model_selection import train_test_split

X = df.copy()
y = X.pop(TARGET_COLS)
kwargs = {
    "test_size": 0.40,
    "random_state": 42,
}
X_train, X_test, y_train, y_test = train_test_split(
    df, 
    y,
    **kwargs
)

In [ ]:
from src.utils.model_utils import get_pipeline

pipeline = get_pipeline(
    ohe_cols=cat_cols,
    num_cols=num_cols,
    boolean_cols=[],
)

from src.utils.model_utils import xgb_param_search
tuning_result = xgb_param_search(
    pipeline=pipeline,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
)

pipeline = tuning_result.best_estimator

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
y_train_pred = pipeline.predict(X_train)
y_test_pred = pipeline.predict(X_test)

In [ ]:
from src.utils.model_utils import evaluate_test_train_models

evaluate_test_train_models(
    y_train=y_train,
    y_train_pred=y_train_pred,
    y_test=y_test,
    y_test_pred=y_test_pred,
).model_dump()

In [ ]:
df_test = X_test.copy()
df_test['y_true'] = y_test
df_test['y_pred'] = y_test_pred
df_test.head(50)
